In [24]:
import os
import re
import sys
import time
import subprocess
import urllib.request
import urllib.error
import json
import csv

from   collections import defaultdict

import pandas as pd
import numpy  as np
from   pandas.io.json import json_normalize
from   pandas import Series, DataFrame

import nltk

import gensim
from   gensim import corpora, models
from   nltk.stem.porter import PorterStemmer
from   nltk import stem
from   nltk.corpus import stopwords

#clustering tools
from   sklearn.feature_extraction.text import TfidfVectorizer
from   sklearn.feature_extraction.text import CountVectorizer
from   sklearn.cluster import KMeans
from   sklearn.cluster import DBSCAN
import hdbscan

import MeCab
m  = MeCab.Tagger("")

import pprint
pp = pprint.PrettyPrinter(indent=4)

本プログラムで使用するライブラリを設定する。

In [25]:
ORG_LIST_FILE      = 'org_name_list.json'
MYLIB_STOPWORD     = 'MyStopWords.txt'
DICTIONARY_FILE    = 'dictionary.dict'
CORPUS_FILE        = 'od_corpus.mm'
LDA_MODEL_FILE     = 'lda_52_sixty_good_model'

NUM_PAGE           = 218
NUM_PER_REQ        = 100

RESULT_HOME_DIR    = '/home/nyoshimatsu/work/odcs/analysis/'
RESULT_G_NAME      = 'g'
RESULT_V_NAME      = 'v'
RESULT_D_NAME      = 'd'
RESULT_M_NAME      = 'm'
WORK_DIR           = '/home/nyoshimatsu/work/odcs/harvest/'
SCRIPT_DIR         = WORK_DIR + 'scripts/topic/'

DIRNAME            = 'default'

RESULT_G_DIR       = RESULT_HOME_DIR + DIRNAME + '/' + RESULT_G_NAME + '/'
RESULT_V_DIR       = RESULT_HOME_DIR + DIRNAME + '/' + RESULT_V_NAME + '/'
RESULT_D_DIR       = RESULT_HOME_DIR + DIRNAME + '/' + RESULT_D_NAME + '/'
RESULT_M_DIR       = RESULT_HOME_DIR + DIRNAME + '/' + RESULT_M_NAME + '/'

プログラム内で使用する定数、ディレクトリ名、ファイル名を設定。

In [26]:
#######################################
# defs
#######################################

def load_mystopwords():
    file_name    = RESULT_V_DIR + MYLIB_STOPWORD
    ff            = open( file_name, 'r' )

    mylib_stopwords = []
    for line in ff:
        line = line.strip()
        mylib_stopwords.append(line)

    ff.close()

    return(mylib_stopwords)

def load_stopwords():
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'

    with urllib.request.urlopen(url) as response:
        slothlib_stopwords = [w for w in response.read().decode().split('\r\n') if w != '']

    return(slothlib_stopwords)

def mash_lists(marged_list):

    mashed_str = marged_list
    
    mashed_str  = mashed_str.replace( "],", "")
    mashed_str  = mashed_str.replace( "]", "")
    mashed_str  = mashed_str.replace( "[", "")
    mashed_str  = mashed_str.replace( "'", "")
    mashed_str  = re.sub( ",\Z", "", mashed_str)
    mashed_list = mashed_str.split(",")

    return(mashed_list)

def calculate_topic_pop(df):
    results_dict = {}
    
    for row_num in df.index:
        for tup in df.topic_comp[row_num]:
            if not tup[0] in results_dict:
                if tup[1] >= 0.1:
                    results_dict[tup[0]] = (tup[1] * (df.iloc[row_num].download_count + 
                                             df.iloc[row_num].page_views_total_log))
                else:
                    pass
            if tup[0] in results_dict:
                if tup[1] >= 0.1:
                    results_dict[tup[0]] += (tup[1] * (df.iloc[row_num].download_count + 
                                             df.iloc[row_num].page_views_total_log))
    return results_dict
  
def clensing_basic(w):
    w = w.replace('（', '')
    w = w.replace('）', '')
    w = w.replace('(' , '')
    w = w.replace(')' , '')
    w = w.replace(',' , '')
    w = w.replace('、', '')
    w = w.replace('，', '')
    w = w.replace('" ', '')
    w = w.replace('"' , '')
    w = w.replace('「', '')
    w = w.replace('」', '')
    return w

def wakachi(content):
    word_list     = []
    wakachi_words = ""

    if not content == '':
        parsed_content = m.parse(content)
        parsed_lines   = re.split('\n', parsed_content)

        for parsed_line in parsed_lines:
            parsed_words1 = re.split('[ \t]+', parsed_line)
            parsed_word   = parsed_words1[0]

            if len(parsed_words1) > 1:
                parsed_words2 = re.split('\,', parsed_words1[1])
                parsed_part   = parsed_words2[0]

               #if parsed_part == '名詞' or parsed_part == '動詞':
                if parsed_part == '名詞':
                    word_list.append(parsed_word)

        wakachi_words = ""
        for word in word_list:
            wakachi_words = wakachi_words + " " + word

    return(wakachi_words)

def find_url(result_dict, org_url_dict, page_data):

    num_results    = len (result_dict['result'])
  
    for i in range(num_results) :
        num_resources      = len (result_dict['result'][i]['resources'])
        num_tags           = len (result_dict['result'][i]['tags'])

        package_author     = result_dict['result'][i]['maintainer']
        if package_author is not None :
            package_author     = package_author.replace('\r\n', '')
            package_author     = package_author.replace('\n',   '')

        organization_name  = result_dict['result'][i]['organization']['name']
        if  organization_name is not None :
            organization_name  = organization_name.replace('\r\n', '')
            organization_name  = organization_name.replace('\n',   '')

        organization_title = result_dict['result'][i]['organization']['title']
        if organization_title is not None :
            organization_title = organization_title.replace('\r\n', '')
            organization_title = organization_title.replace('\n',   '')

        dataset_desc_list  = []
        dataset_desc       = result_dict['result'][i]['notes']
        if dataset_desc is not None :
            dataset_desc     = dataset_desc.replace(',',    '')
            dataset_desc     = dataset_desc.replace('\r\n', '')
            dataset_desc     = dataset_desc.replace('\n',   '')
            dataset_desc     = dataset_desc.strip()

            dataset_desc_sep = wakachi(dataset_desc)
            dataset_desc     = clensing_basic(dataset_desc_sep)
 
            dataset_desc_list = re.split('[ ]+', dataset_desc)

        dataset_name = result_dict['result'][i]['name']
        if dataset_name is not None :
            dataset_name = dataset_name.replace('\r\n', '')
            dataset_name = dataset_name.replace('\n',   '')
 
        group_list   = []
        group_domain = ''

        tag_list     = []
        for j in range(num_tags):
            tag        = result_dict['result'][i]['tags'][j]['name']
            tag_list.append(tag)

        # domain_name = result_dict['result'][i]['url']
        domain_name = org_url_dict[organization_name]['url']

        mash_list    = []
        big_mash_list= []

        download_count        = 1
        page_views_last_month = 1
        page_views_last_week  = 1
        page_views_total      = 1
        page_views_total_log  = 1
            
        # update dict data
        rsc_data = pd.Series({
            'name':                  dataset_name,
            'description':           dataset_desc_list,
            'attribution':           package_author,
            'column_field_name':     '',
            'columns_name':          '',
            'type':                  'file',
            'categories':            group_list,
            'domain_category':       group_domain,
            'domain_tags':           tag_list,
            'provenance':            'official',
            'download_count':        download_count,
            'page_views_last_month': page_views_last_month,
            'page_views_last_week':  page_views_last_week,
            'page_views_total':      page_views_total,
            'page_views_total_log':  page_views_total_log,
            'domain':                domain_name,
            'mash':                  mash_list,
            'big_mash':              big_mash_list
        })
    
        page_data = page_data.append(rsc_data, ignore_index=True)

    return (page_data)


本プログラムで使用する関数の定義。

In [27]:
file_name    = RESULT_V_DIR + ORG_LIST_FILE
f            = open( file_name, 'r' )
org_url_dict = json.load(f)

自治体名と自治体コード、および、オープンデータサイトの対応表を読み込み。

In [28]:
result_dict  = {}

pd_data_init = DataFrame(index=[], columns=[
    'name',
    'description',
    'attribution',
    'column_field_name',
    'columns_name',
    'type',
    'categories',
    'domain_category',
    'domain_tags',
    'provenance',
    'download_count',
    'page_views_last_month',
    'page_views_last_week',
    'page_views_total',
    'page_views_total_log',
    'domain',
    'mash',
    'big_mash'
])

pd_data = pd_data_init

print('Loading pages... ')

for i in range(NUM_PAGE):

    print( " %3d" % (i), end="" )
  
    file_name   = RESULT_G_DIR + 'resource_' + str(i)
    f           = open( file_name, 'r' )
    result_json = json.load(f)
    f.close()

    # Check the contents of the response.
    assert result_json['success'] is True
    result_dict = result_json
  
    # check_dict(i, result_dict, org_url_dict)

    page_data = pd_data_init

    page_data = find_url(result_dict, org_url_dict, page_data)

    if len(page_data) > 0:
        pd_data = pd_data.append(page_data)

print(" Done")

pd_data = pd_data.reset_index(drop=True)

read_df = pd_data.copy()

Loading pages... 
   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 Done


事前にharvestから取得したCKANデータを、pandasのデータフレームへ読み込む

In [29]:
df             = read_df.copy()
df['mash']     = df['domain_tags']
df['big_mash'] = df['domain_tags'] + df['description']

トピックモデルでは、CKANのリソースに付加されている'domain_tags'、および、'domain_tags'と'description'を合わせてた文字列を文書と見立てトピックモデルを適用する。
データフレームに、分析に用いるカラムとして'mash'、および、'big_mash'を作成する。

In [30]:
df = df[df.astype(str).mash != "[]"] # drop all blank lists
df = df.reset_index(drop=True)

big_mash = df.copy()

print("len(df):", len(df))

len(df): 6117


'mash'のカラムで、データが無い列を削除する。

In [31]:
mash       = df.big_mash
slothlib_stopwords = load_stopwords()
mylib_stopwords = load_mystopwords()
  
stop_words = stopwords.words('english')
stop       = list(stop_words)
stop.append(slothlib_stopwords)
stop.extend(['&',''])
stop.extend(mylib_stopwords)
stop.extend(np.arange(101).astype(str))
stop.extend(np.arange(1980, 2025).astype(str))
    
texts = []

for lls in mash:
    ls = []
    for ll in lls:
        ll = ll.strip()
        ls.append(ll)
      
    lowers = [word.lower() for word in ls]
    stopped_tokens = [word for word in ls if not word in stop]
    texts.append(stopped_tokens)

stopwordsの機能を利用し、LDAのモデルから外したい単語を除く。

In [32]:
dictionary      = corpora.Dictionary(texts)
dictionary_file = RESULT_M_DIR + DICTIONARY_FILE
dictionary.save(dictionary_file)

LDAのモデル作成で使用する辞書（各単語に対し番号を割当た対応表）を作成する。

In [33]:
corpus          = [dictionary.doc2bow(text) for text in texts]

corpus_filename = RESULT_M_DIR + CORPUS_FILE
corpora.MmCorpus.serialize(corpus_filename, corpus)

LDAのモデル作成で使用するコーパス（単語IDと単語の出現頻度表）を作成する。

In [34]:
topic_N = 20
pass_N  = 60

print(topic_N," topics", pass_N, " passes")

lda_52_sixty = gensim.models.ldamodel.LdaModel(
    corpus    = corpus,
    num_topics= topic_N,
    passes    = pass_N,
    id2word   = dictionary,
    random_state = 7)

lda_model = lda_52_sixty.show_topics(num_topics=topic_N, formatted=False)

pp.pprint(lda_model)

20  topics 60  passes
[   (   0,
        [   ('施設', 0.12758538442331702),
            ('位置情報', 0.10602021057806799),
            ('子ども', 0.053730517008294496),
            ('施設情報', 0.051643867009916911),
            ('概要', 0.047763065326178009),
            ('学校', 0.027141548716634802),
            ('学習', 0.018771338640216371),
            ('病院', 0.016334271939516205),
            ('経度', 0.01320536245973984),
            ('緯度', 0.01320536245973984)]),
    (   1,
        [   ('調査', 0.050259836666825174),
            ('資料', 0.03582394049239887),
            ('企業', 0.035465103707183011),
            ('工業', 0.029723648100512701),
            ('事業所', 0.026217864607823157),
            ('各種', 0.025332453963560847),
            ('情報', 0.024925753782987631),
            ('地図', 0.024196170074210432),
            ('目的', 0.021548363145385944),
            ('4月1日', 0.019588171403801886)]),
    (   2,
        [   ('子育て', 0.11192938879439424),
            ('施設', 0.043067663067310122),
            ('

LDAのモデルを生成する

In [23]:
lda_model_filename = RESULT_M_DIR + LDA_MODEL_FILE
lda_52_sixty.save(lda_model_filename)

LDAのモデルを保存する。